# 
# HyTEST hydrologic model benchmark assessment: visualization

**Last Update, June 2022**

This notebook is an example of how a HyTEST user may examine streamflow benchmark results from a hydrologic model. Here, we are viewing daily streamflow benchmark results from the [National Water Model Retrospective version 2.1](https://registry.opendata.aws/nwm-archive/), forced with AORC, at streamflow benchmark locations (version 1), "cobalt gages" ([Foks et al., 2022](https://www.sciencebase.gov/catalog/item/6181ac65d34e9f2789e44897)).

Two benchmark results are examined, the standard statistical suite results ([Towler et al., 2022](https://www.sciencebase.gov/catalog/item/62336af9d34ec9f19eeb48fd)) and the decomposition statistical suite (d-score; [Hodson et al., 2022](https://www.sciencebase.gov/catalog/item/61d4c9e9d34ed79293fe91b4)).

In [ ]:
# Prior to beginning, ensure that following Python librariers are installed and loaded
import xarray as xr
import pandas as pd
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import panel as pn
from geoviews import tile_sources as gvts
import intake
import param
import cartopy.crs as ccrs

In [ ]:
## Import daily streamflow benchmark results from intake catalog
## feel free to try the below code on HPC or cloud resources to view datasets avaliable on the intake catalog
#url = 'https://raw.githubusercontent.com/USGS-python/hytest-catalogs/main/hytest_intake_catalog.yml'
#cat = intake.open_catalog(url)
#list(cat)

In [ ]:
# load the following datasets from github since this has not been updated for the intake catalog. This is not ideal as data gets larger and users start to clone the repo.

# streamflow site list (Cobalt gages)
df0 = pd.read_csv('https://raw.githubusercontent.com/USGS-python/hytest-evaluation-workflows/main/misc/streamflow_gages_v1_n5390.csv', dtype={'site_no': str})

#daily streamflow benchmark results for NWM v2.1 - the standard suite of metrics v1.0
df1 = pd.read_csv('https://raw.githubusercontent.com/USGS-python/hytest-evaluation-workflows/main/misc/standard_suite_v1_nwmv2d1.csv', dtype={'site_no': str})

#daily streamflow benchmark results for NWM v2.1 - the decomposition suite v0.1
df2 = pd.read_csv('https://raw.githubusercontent.com/USGS-python/hytest-evaluation-workflows/main/misc/streamflow_nwm_v2.1-dscore_v0.1-benchmark_v1.csv', dtype={'site_no': str})

In [ ]:
## rename lat, lon for plotting
df0 = df0.rename(columns={'dec_lat_va':'Lat', 'dec_long_va':'Lon'} )

## merge site location information with benchmark results
df1 = pd.merge(df1, df0)

## Benchmark results over the spatial extent of the conterminous United States

In [ ]:
## df1 is presented here, and specific columns are called out. If wanting to modify, please see code below.
## define which columns are metrics in the widget
var_select = pn.widgets.Select(name='Metric', options=list(df1.columns[1:-15]), 
                               value='pearson_r')

base_map_select = pn.widgets.Select(name='Basemap:', 
                                    options=list(gvts.tile_sources.keys()), 
                                    value='OSM')

@pn.depends(var_select, base_map_select)

def plot(var, base_map):
    return df1.hvplot.points(x='Lon', y='Lat', color=var, cmap='turbo_r', geo=True, tiles=base_map)

col = pn.Column(var_select, base_map_select, plot)
col.servable('Hydro Assessment Tool')

## Summary of benchmark statistics

### Boxplots GAGES-II classification, HUC02 group, or aggregated ecoregions

In [ ]:
# define which columns are metrics in the widget and which ones are groups
var_select = pn.widgets.Select(name='Metric', options=list(df1.columns[1:-15]), 
                               value='r')

group_select = pn.widgets.Select(name='Group By:', 
                                    options=list(df1.columns[20:-6]), 
                                    value='aggecoregion')

@pn.depends(var_select, group_select)

def plot(var, group):
    return df1.hvplot.box(y = var, by = group, height=400, width=800, legend=False)

col = pn.Column(var_select, group_select, plot)
col.servable('boxplots')

## Future additions: 
## Hover over box to tell user exactly the number of samples in group (count), median, mean, max, min, and IQR.
## Add: different states, shapefile user-upload, HLRs, west vs east conus, IWS basins (and subbasins).

### Histograms by GAGES-II classification, HUC02 group, or aggregated ecoregions

In [ ]:
# define which columns are metrics in the widget and which ones are groups
var_select = pn.widgets.Select(name='Metric', options=list(df1.columns[1:-15]), 
                               value='r')

group_select = pn.widgets.Select(name='Group By:', 
                                    options=list(df1.columns[20:-6]), 
                                    value='aggecoregion')

@pn.depends(var_select, group_select)

def plot(var, group):
    return df1.hvplot.hist(var, group, subplots=True, width=400, bins = 500, legend='top')

col = pn.Column(var_select, group_select, plot)
col.servable('histograms')

### Metric by Latitude & Longitude

In [ ]:
# define which columns are metrics in the widget and which ones are groups
var_select = pn.widgets.Select(name='Metric', options=list(df1.columns[1:-15]), 
                               value='r')

group_select = pn.widgets.Select(name='Group By:', 
                                    options=list(df1.columns[14:16]), 
                                    value='Lon')

@pn.depends(var_select, group_select)

def plot(var, group):
    return df1.hvplot.scatter(x=group, y=var, height=400, width = 500, legend='top', hover_cols=["site_no","Lat","Lon"])

col = pn.Column(var_select, group_select, plot)
col.servable('scatter1')

### Metric v. Metric

In [ ]:
# define which columns are metrics in the widget and which ones are groups
var_select = pn.widgets.Select(name='Metric', options=list(df1.columns[1:-15]), 
                               value='r')

var2_select = pn.widgets.Select(name='Metric:', 
                                    options=list(df1.columns[1:-15]), 
                                    value='rSpearman')

@pn.depends(var_select, var2_select)

def plot(var, var2):
    return df1.hvplot.scatter(x = var, y = var2, height=400, width = 500, legend='top', hover_cols=['site_no','Lat', 'Lon'])

col = pn.Column(var_select, var2_select, plot)
col.servable('scatter2')

## Browse Table of Raw Statistics

### Standard statistical suite

In [ ]:
# scrollable table with filtering mechanisms
df1.hvplot.table(columns=['site_no', 'drain_sqkm','KGE', 'NSE','logNSE','r','rSpearman','rSD','PBIAS','pbiasfdc','PBIAS_HF','PBIAS_LF','complete_yrs','n_days'], sortable=True, selectable=True)

### Decomposition statistical suite (d-score)

In [ ]:
# scrollable table with filtering mechanisms
df2 = pd.merge(df2, df0)
df2.hvplot.table(columns=['site_no', 'drain_sqkm','overall','trend','seasonality','variability','bias','distribution','sequence','winter','spring','summer','fall','low','below_avg','above_avg','high','complete_yrs','n_days'], sortable=True, selectable=True)

## References

Foks, S.S., Towler, E., Hodson, T.O., Bock, A.R., Dickinson, J.E., Dugger, A.L., Dunne, K.A., Essaid, H.I., Miles, K.A., Over, T.M., Penn, C.A., Russell, A.M., Saxe, S.W., and Simeone, C.E., 2022, Streamflow benchmark locations for conterminous United States (cobalt gages): U.S. Geological Survey data release, https://doi.org/10.5066/P972P42Z

Towler, E., Foks, S.S., Dickinson, J.E., Dugger, A.L., Essaid, H.I., Gochis, D., Hodson, T.O., Viger, R.J., and Zhang, Y., 2022, Daily streamflow performance benchmark defined by the standard statistical suite (v1.0) for the National Water Model Retrospective (v2.1) at benchmark streamflow locations: U.S. Geological Survey data release, https://doi.org/10.5066/P9QT1KV7

## *coming soon* - Benchmark results scorecard
The scorecard features the results of the decomposition statistical suite, 'd-score'. Cooler colors represent smaller relative error between the model and the evaluation data, whereas warmer colors represent larger error ([Hodson et al., 2021](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021MS002681)). The dscore is ideal when examining or comparing the performance of two or more hydrologic models, though running dscore for one hydrologic model can tell us how much errors are associated with different components within a decomposition. 

#### References:

Hodson, T.O., Foks, S.S., Dugger, A.L., Dunne, K.A., Miles, K.A., Over, T.M., Penn, C.A., Saxe, S.W., Simeone, C.E., Towler, E., and Viger, R.J., 2022, Daily streamflow performance benchmark defined by D-score (v0.1) for the National Water Model (v2.1) at benchmark streamflow locations: U.S. Geological Survey data release, https://doi.org/10.5066/P9MJDNRL

Hodson, T.O., Over, T.M. and Foks, S.S., 2021. Mean squared error, deconstructed. Journal of Advances in Modeling Earth Systems, 13(12), p.e2021MS002681, https://doi.org/10.1029/2021MS002681